In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras import models
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [67]:
train = pd.read_csv("/train.csv")
test = pd.read_csv("/test.csv")
#train.isnull().sum()
train.info()     
comb = pd.concat([train, test], axis=0, sort=False)

#comb.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [68]:
train['Street'].value_counts()
print(comb[comb['PoolQC'].notnull()]['PoolQC'])

197     Ex
810     Fa
1170    Gd
1182    Ex
1298    Gd
1386    Fa
1423    Gd
514     Ex
1113    Ex
1250    Gd
Name: PoolQC, dtype: object


In [69]:
#update the duplicate values 
numeric_var_list = ['LotFrontage','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','GarageYrBlt','GarageCars','GarageArea']
categorical_val_list = ['MSZoning','Alley','Utilities','Exterior1st','Exterior2nd','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','KitchenQual','SaleType','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

#update the missed numaric values by mean

#comb['LotFrontage'] = comb['LotFrontage'].fillna(comb['LotFrontage'].mean())

for nv in numeric_var_list:
  comb[nv] = comb[nv].fillna(comb[nv]).median()
for cv in categorical_val_list:
  comb[cv] = comb[cv].fillna(comb[cv]).mode()[0]
  

for c,type in zip(comb.columns,comb.dtypes):
  if(type=='object'):
    temp1=pd.get_dummies(comb[c],drop_first=True)
    comb = pd.concat([comb,temp1],axis=1)
    comb.drop([c],axis=1,inplace=True) 

#Remove the duplicate columns    
comb = comb.loc[:,~comb.columns.duplicated()]    
  
  
#comb.isnull().sum()  
#comb.info()

#1460

#df_Train=final_df.iloc[:1422,:]
#df_Test=final_df.iloc[1422:,:]
train_x = comb.iloc[:1460,:]
test_x =  comb.iloc[1460:,:]
train_y = train_x[train_x['SalePrice'].notnull()]['SalePrice']

train_x.drop(['SalePrice'],axis=1,inplace=True)
test_x.drop(['SalePrice'],axis=1,inplace=True)

print(train_x.shape)
print(test_x.shape)

(1460, 126)
(1459, 126)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [70]:
#SPlit the categorized values and concat with the initial values
#lab_dtype = train_x.dtypes
#train_x.dtypes
#train_x.dtypes

'''
for c,type in zip(train_x.columns,train_x.dtypes):
  if(type=='object'):
    temp1=pd.get_dummies(train_x[c],drop_first=True)
    train_x = pd.concat([train_x,temp1],axis=1)
    train_x.drop([c],axis=1,inplace=True)

    
for c,type in zip(test_x.columns,test_x.dtypes):
    temp1=pd.get_dummies(test_x[c],drop_first=True)
    test_x = pd.concat([test_x,temp1],axis=1)
    test_x.drop([c],axis=1,inplace=True)
    
'''

"\nfor c,type in zip(train_x.columns,train_x.dtypes):\n  if(type=='object'):\n    temp1=pd.get_dummies(train_x[c],drop_first=True)\n    train_x = pd.concat([train_x,temp1],axis=1)\n    train_x.drop([c],axis=1,inplace=True)\n\n    \nfor c,type in zip(test_x.columns,test_x.dtypes):\n    temp1=pd.get_dummies(test_x[c],drop_first=True)\n    test_x = pd.concat([test_x,temp1],axis=1)\n    test_x.drop([c],axis=1,inplace=True)\n    \n"

In [71]:
#Do the training operation from the resepctive features
#Linear Regression 

#Train the model

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from math import sqrt
model = linear_model.LinearRegression()
model.fit(train_x, train_y)

predicted_train_y = model.predict(train_x)
#for a,p in zip(train_y,predicted_train_y):
#  print(a," ",p)
predicted_y = model.predict(test_x)
print(len(predicted_train_y))

print("The RMSE Error is ",sqrt(mean_squared_error(predicted_train_y,train_y)))
#print("Accuracy --> ", model.score(train_x,predicted_train_y)*100)
#print("Predict value " + str(model.predict([test_x[142]])))

1460
The RMSE Error is  26795.18356335712


In [72]:
#Perform the operation using Random forest
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=1000)
model.fit(train_x, train_y)

predicted_train_y = model.predict(train_x)
#for a,p in zip(train_y,predicted_train_y):
#  print(a," ",p)
predicted_y = model.predict(test_x)
print("The RMSE Error is ",sqrt(mean_squared_error(predicted_train_y,train_y)))


The RMSE Error is  11083.808267791283


In [73]:
#Perform gradient boosting 
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=1000, max_depth=6)
model.fit(train_x, train_y)

predicted_train_y = model.predict(train_x)
#for a,p in zip(train_y,predicted_train_y):
#  print(a," ",p)
predicted_y = model.predict(test_x)
print("The RMSE Error is ",sqrt(mean_squared_error(predicted_train_y,train_y)))

The RMSE Error is  48.21217805121048


In [74]:
#parameter optimization

print( train_x.columns[train_x.columns.duplicated()])
print( test_x.columns[test_x.columns.duplicated()])

from sklearn.model_selection import RandomizedSearchCV
import xgboost

classifier=xgboost.XGBRegressor()
regressor=xgboost.XGBRegressor()

booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }
# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)
random_cv.fit(train_x, train_y)


Index([], dtype='object')
Index([], dtype='object')
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   27.5s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  4.2min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  8.4min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 12.9min finished
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[09:51:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_st...
                   iid='warn', n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree',

In [75]:
random_cv.best_estimator_

XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=900,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [78]:

model=xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=900,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

model.fit(train_x,train_y)
predicted_train_y = model.predict(train_x)
#for a,p in zip(train_y,predicted_train_y):
#  print(a," ",p)
predicted_y = model.predict(test_x)
print("The RMSE Error is ",sqrt(mean_squared_error(predicted_train_y,train_y)))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[09:57:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The RMSE Error is  12592.983513566038


In [79]:
#using Neural network to minimize the error
# Create the Model
model = models.Sequential()
model.add(layers.Dense(32, input_dim=train_x.shape[1], activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))

# Compile the Model
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=["mae"])

model.fit(train_x,train_y, epochs=200, batch_size=32)
predicted_train_y = model.predict(train_x)
#for a,p in zip(train_y,predicted_train_y):
#  print(a," ",p)
predicted_y = model.predict(test_x)
print("The RMSE Error is ",sqrt(mean_squared_error(predicted_train_y,train_y)))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1168 samples, validate on 292 samples
Epoch 1/200





1168/1168 [==============================] - 1s 712us/step - loss: 144.6871 - mean_absolute_error: 180864.8519 - val_loss: 144.9388 - val_mean_absolute_error: 182514.2517
Epoch 2/200
1168/1168 [==============================] - 0s 56us/step - loss: 144.6871 - mean_absolute_error: 180864.8523 - val_loss: 144.9388 - val_mean_absolute_error: 182514.2517
Epoch 3/200
1168/1168 [==============================] - 0s 49us/step - loss: 144.6871 - mean_absolute_error: 180864.8527 - val_loss: 144.9388 - val_mean_absolute_error: 182514.2517
Epoch 4/200
1168/1168 [==============================] - 0s 52us/step - loss: 144.6871 - mean_absolute_error: 180864.8532 - val_loss: 144.9388 - val_mean_absolute_error: 182514.2517
Epoch 5/200
1168/1168 [==============================] - 0s 49us/step - loss: 144.6871 - mean_absolute_error: 180864.